## 1. Setup and Configuration

In [ ]:
import os
import time
import json
import requests
from typing import Dict

# Modern data stack
import psycopg
import polars as pl
import plotly.express as px
import plotly.graph_objects as go

# Configuration
DB_CONFIG = {
    "host": os.getenv("POSTGRES_HOST", "postgres.postgres.svc.cluster.local"),
    "port": os.getenv("POSTGRES_PORT", "5432"),
    "user": os.getenv("POSTGRES_USER", "demo"),
    "password": os.getenv("POSTGRES_PASSWORD", "demo"),
    "dbname": os.getenv("POSTGRES_DB", "demo_noetl")
}

NOETL_SERVER_URL = os.getenv(
    "NOETL_SERVER_URL", 
    "http://noetl.noetl.svc.cluster.local:8082"
)

TEST_PATH = "tests/pagination/loop_with_pagination"
POLL_INTERVAL = 2
MAX_WAIT = 120

print("✓ Configuration loaded")
print(f"  Server: {NOETL_SERVER_URL}")
print(f"  Database: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}")
print(f"  Test: {TEST_PATH}")

## 2. Initialize Test Table

In [ ]:
# Create test schema and table - completely self-contained
import os
import psycopg

def get_postgres_connection():
    """Get psycopg3 connection"""
    conn_string = f"host={os.getenv('POSTGRES_HOST', 'postgres.postgres.svc.cluster.local')} " \
                  f"port={os.getenv('POSTGRES_PORT', '5432')} " \
                  f"dbname={os.getenv('POSTGRES_DB', 'demo_noetl')} " \
                  f"user={os.getenv('POSTGRES_USER', 'demo')} " \
                  f"password={os.getenv('POSTGRES_PASSWORD', 'demo')}"
    return psycopg.connect(conn_string)

create_table_sql = """
CREATE SCHEMA IF NOT EXISTS noetl_test;

DROP TABLE IF EXISTS noetl_test.pagination_loop_results;

CREATE TABLE noetl_test.pagination_loop_results (
    id SERIAL PRIMARY KEY,
    execution_id BIGINT,
    endpoint_name TEXT,
    endpoint_path TEXT,
    page_size INTEGER,
    result_count INTEGER,
    result_data JSONB,
    iteration_index INTEGER,
    iteration_count INTEGER,
    created_at TIMESTAMP DEFAULT NOW()
);

CREATE INDEX IF NOT EXISTS idx_pagination_loop_execution_id 
ON noetl_test.pagination_loop_results(execution_id);
"""

with get_postgres_connection() as conn:
    with conn.cursor() as cur:
        cur.execute(create_table_sql)
        conn.commit()

print("✓ Test table created")
print("  Schema: noetl_test")
print("  Table: pagination_loop_results")

## 3. Database Utilities

In [ ]:
def get_postgres_connection():
    """Get psycopg3 connection"""
    conn_string = f"host={DB_CONFIG['host']} port={DB_CONFIG['port']} " \
                  f"dbname={DB_CONFIG['dbname']} user={DB_CONFIG['user']} " \
                  f"password={DB_CONFIG['password']}"
    return psycopg.connect(conn_string)

def query_to_polars(query: str) -> pl.DataFrame:
    """Execute query and return as Polars DataFrame"""
    with get_postgres_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            columns = [desc[0] for desc in cur.description]
            data = cur.fetchall()
    if not data:
        return pl.DataFrame(schema=columns)
    return pl.DataFrame({col: [row[i] for row in data] for i, col in enumerate(columns)})

print("✓ Database utilities loaded")

## 4. Execute Pagination Loop Test

In [ ]:
def start_test() -> Dict:
    """Start pagination loop test"""
    url = f"{NOETL_SERVER_URL}/api/run/playbook"
    payload = {"path": TEST_PATH}
    
    print(f"Starting test: {TEST_PATH}")
    response = requests.post(url, json=payload, timeout=30)
    response.raise_for_status()
    
    result = response.json()
    execution_id = result['execution_id']
    
    print(f"✓ Test started")
    print(f"  Execution ID: {execution_id}")
    print(f"  Status: {result['status']}")
    
    return result

test_result = start_test()
EXECUTION_ID = test_result['execution_id']

## 5. Monitor Execution

In [ ]:
def monitor_execution(execution_id: int):
    """Monitor test execution"""
    start_time = time.time()
    last_count = 0
    
    print(f"Monitoring execution {execution_id}...")
    print(f"{'Time':<6} {'Steps':<6} {'Status':<12} {'Events'}")
    print("-" * 50)
    
    while (time.time() - start_time) < MAX_WAIT:
        query = f"""
            SELECT event_type, COUNT(*) as count
            FROM noetl.event
            WHERE execution_id = {execution_id}
            GROUP BY event_type
        """
        df = query_to_polars(query)
        
        step_count = df.filter(pl.col('event_type') == 'step_completed')['count'].sum() or 0
        is_complete = df.filter(pl.col('event_type') == 'playbook_completed').height > 0
        is_failed = df.filter(pl.col('event_type') == 'playbook_failed').height > 0
        
        if step_count != last_count or is_complete or is_failed:
            elapsed = int(time.time() - start_time)
            status = "COMPLETED" if is_complete else ("FAILED" if is_failed else "RUNNING")
            total = df['count'].sum()
            print(f"{elapsed:<6} {step_count:<6} {status:<12} {total}")
            last_count = step_count
        
        if is_complete:
            print(f"\n✓ Test completed in {elapsed}s")
            return True
        elif is_failed:
            print(f"\n✗ Test failed after {elapsed}s")
            return False
        
        time.sleep(POLL_INTERVAL)
    
    print(f"\n⚠ Timeout after {MAX_WAIT}s")
    return False

success = monitor_execution(EXECUTION_ID)

## 6. Debug: Check Execution Events

In [ ]:
# Check execution events for errors
events_query = f"""
    SELECT 
        event_type,
        node_name,
        status,
        message,
        created_at
    FROM noetl.event
    WHERE execution_id = {EXECUTION_ID}
    ORDER BY created_at
"""

events_df = query_to_polars(events_query)

print(f"\n📋 Execution Events for {EXECUTION_ID}:")
print("=" * 80)
print(events_df)

# Check for failures
failed_events = events_df.filter(
    (pl.col('status') == 'failed') | 
    (pl.col('event_type').str.contains('failed'))
)

if failed_events.height > 0:
    print("\n❌ Failed Events:")
    print(failed_events)
    
    # Show error messages
    for row in failed_events.iter_rows(named=True):
        print(f"\nNode: {row['node_name']}")
        print(f"Error: {row['message']}")
else:
    print("\n✓ No failed events found")

## 7. Analyze Pagination Data

In [ ]:
# Get saved pagination results
pagination_query = f"""
    SELECT 
        endpoint_name,
        endpoint_path,
        page_size,
        result_count,
        iteration_index,
        iteration_count,
        created_at
    FROM noetl_test.pagination_loop_results
    WHERE execution_id = {EXECUTION_ID}
    ORDER BY iteration_index
"""

try:
    pagination_df = query_to_polars(pagination_query)
    
    print("\n📊 Pagination Results:")
    print(pagination_df)
    
    print(f"\n📈 Summary:")
    print(f"  Total endpoints: {pagination_df.height}")
    print(f"  Total items fetched: {pagination_df['result_count'].sum()}")
    print(f"  Endpoints: {', '.join(pagination_df['endpoint_name'].to_list())}")
except Exception as e:
    if "does not exist" in str(e):
        print("⚠️  Table 'noetl_test.pagination_loop_results' does not exist")
        print("   Run cell 3 (Initialize Test Table) first!")
    else:
        raise

## 8. Validate Results

In [ ]:
print("\n" + "="*60)
print("VALIDATION REPORT")
print("="*60)

validation = {'passed': True, 'issues': []}

# Check 1: Two endpoints tested
if pagination_df.height == 2:
    print("\n✓ Check 1: Tested 2 endpoints")
else:
    print(f"\n✗ Check 1: Expected 2 endpoints, got {pagination_df.height}")
    validation['passed'] = False
    validation['issues'].append(f"Endpoint count mismatch: {pagination_df.height}")

# Check 2: Assessments endpoint
assessments = pagination_df.filter(pl.col('endpoint_name') == 'assessments')
if assessments.height > 0:
    count = assessments['result_count'][0]
    if count == 35:
        print(f"✓ Check 2: Assessments fetched {count} items")
    else:
        print(f"✗ Check 2: Expected 35 assessments, got {count}")
        validation['passed'] = False
        validation['issues'].append(f"Assessments count: {count}")
else:
    print("✗ Check 2: Assessments endpoint not found")
    validation['passed'] = False
    validation['issues'].append("Missing assessments endpoint")

# Check 3: Users endpoint
users = pagination_df.filter(pl.col('endpoint_name') == 'users')
if users.height > 0:
    count = users['result_count'][0]
    if count == 35:
        print(f"✓ Check 3: Users fetched {count} items")
    else:
        print(f"✗ Check 3: Expected 35 users, got {count}")
        validation['passed'] = False
        validation['issues'].append(f"Users count: {count}")
else:
    print("✗ Check 3: Users endpoint not found")
    validation['passed'] = False
    validation['issues'].append("Missing users endpoint")

# Check 4: Total items
total = pagination_df['result_count'].sum()
if total == 70:
    print(f"✓ Check 4: Total items fetched: {total}")
else:
    print(f"✗ Check 4: Expected 70 total items, got {total}")
    validation['passed'] = False
    validation['issues'].append(f"Total count: {total}")

# Check 5: Iteration indices
indices = pagination_df['iteration_index'].to_list()
if indices == [0, 1]:
    print(f"✓ Check 5: Iteration indices correct: {indices}")
else:
    print(f"✗ Check 5: Expected [0, 1], got {indices}")
    validation['passed'] = False
    validation['issues'].append(f"Iteration indices: {indices}")

# Final status
print("\n" + "="*60)
if validation['passed']:
    print("🎉 ALL CHECKS PASSED")
else:
    print("❌ VALIDATION FAILED")
    print("\nIssues:")
    for issue in validation['issues']:
        print(f"  - {issue}")
print("="*60)

## 9. Visualize Performance

In [ ]:
# Bar chart: Items per endpoint
if pagination_df.height > 0:
    fig = px.bar(
        pagination_df.to_pandas(),
        x='endpoint_name',
        y='result_count',
        title=f'Items Fetched per Endpoint - Execution {EXECUTION_ID}',
        labels={'endpoint_name': 'Endpoint', 'result_count': 'Items Count'},
        text='result_count'
    )
    fig.update_traces(textposition='outside')
    fig.add_hline(y=35, line_dash="dash", annotation_text="Expected: 35")
    fig.show()

## 10. Event Timeline

In [ ]:
# Get execution events
events_query = f"""
    SELECT 
        created_at,
        event_type,
        node_name,
        status
    FROM noetl.event
    WHERE execution_id = {EXECUTION_ID}
    ORDER BY created_at
"""

events_df = query_to_polars(events_query).to_pandas()

if len(events_df) > 0:
    fig = px.scatter(
        events_df,
        x='created_at',
        y='event_type',
        color='status',
        hover_data=['node_name'],
        title=f'Event Timeline - Execution {EXECUTION_ID}',
        labels={'created_at': 'Time', 'event_type': 'Event Type'}
    )
    fig.update_layout(height=600)
    fig.show()

## 11. Cleanup

In [ ]:
# Optional: Clean up test data
cleanup = input("Clean up test data? (y/n): ")

if cleanup.lower() == 'y':
    with get_postgres_connection() as conn:
        with conn.cursor() as cur:
            cur.execute(f"""
                DELETE FROM noetl_test.pagination_loop_results
                WHERE execution_id = {EXECUTION_ID}
            """)
            conn.commit()
    print(f"✓ Cleaned up test data for execution {EXECUTION_ID}")
else:
    print("Test data preserved")

---

## Summary

This notebook validates the `test_loop_with_pagination` playbook:
- ✅ Executes pagination loop test
- ✅ Monitors real-time progress
- ✅ Validates result counts (2 endpoints × 35 items)
- ✅ Analyzes saved pagination data
- ✅ Visualizes performance metrics

**Key Metrics:**
- Expected Endpoints: 2 (assessments, users)
- Expected Items per Endpoint: 35
- Expected Total Items: 70
- Expected Database Records: 2